In [3]:
from land_frontend import movement
from land_frontend import clean_movement
import dijkstra_methods as dk
from datetime import datetime
from datetime import timedelta
import time

#helping method to determine if a patrol is hit
def isin(pattern, seq):
    for i in range(len(seq) - len(pattern) + 1):
        if seq[i:i+len(pattern)] == pattern:
            return True
    return False


#initialise possible patrol list
patrol_list = []
#read patrol database
filename = './sample_patrols.txt'
with open(filename) as fhandle:
    for line in fhandle:
        #print(line)
        p1,p2,desc = line.split('/')
        patrol_list.append([p1,p2,desc])
#print(patrol_list)



def patrol_sweep(speed,start,end,sdatetime='now'):
    #set datetime
    if sdatetime=='now':
        sdatetime = datetime.now()
    else:
        sdatetime = datetime.strptime(sdatetime,'%d/%m/%y %H:%M:%S')
    path,time,dlta = movement(speed,start,end)
    print('checking for patrols--------------------------------------------------------')
    for patrol in patrol_list:
        desc = patrol[2]
        coordinates = patrol[0:2]
        #cut one tile patrols to the appropriate length
        if coordinates[1]=='0':
            coordinates=[coordinates[0]]
        #catch forward patrols
        if isin(coordinates,list(path)): 
            #calculate time delta and end time for a tripped patrol
            pth,endtime,dlta = clean_movement(speed,start,str(coordinates[-1]),sdatetime)
            print('patrol tripped at ',coordinates,' after ',str(dlta),' hours, at ',endtime)
            print(desc)
        #catch reverse patrols
        elif isin(coordinates[::-1],list(path)):
            #calculate time delta and end time for a tripped patrol
            pth,endtime,dlta = clean_movement(speed,start,str(coordinates[-1]),sdatetime)
            print('patrol tripped at ',coordinates,' after ',str(dlta),' hours, at ',endtime)
            print(desc)
patrol_sweep(10,'StormsEnd','S2')

Total hours of travel is  6.0
end time is:  2023-06-13 01:06:19.602895
deque(['StormsEnd', 'S11', 'S7', 'S6', 'S2'])
checking for patrols--------------------------------------------------------
patrol tripped at  ['S2', 'S6']  after  4.8  hours, at  2023-06-12 23:54:19.602883
500 buckler army attacking
